**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Direct Method for Calculating Astronomical Position

This program demonstrates the implementation of the direct method for calculating the observer's position without assumed observer position. The calculation requires data from two celestial bodies, and with the help of a third celestial body, the correct observer's position is determined.

To perform the calculation, we need the following data for each celestial body:
- Height
- Declination
- Greenwich Hour Angle (GHA)

You can determine the declination and Greenwich Hour Angle using a program located in this directory: **[visible_celestial_objects.ipynb](https://barakuda.fpp.uni-lj.si/user/sandro/lab/tree/classes/oce-nav/git-src/celestial_navigation/visible_celestial_objects.ipynb)**

<hr>

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})
import mpl_toolkits.basemap as bmap


import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
# Draw circle path on a Sphere

def circle_pts(fi, la, z, N):
    
    # convert degrees to radians
    fi = nt.deg2rad(fi)
    la = nt.deg2rad(la)
    z = nt.deg2rad(z)
    
    # calculate max Fi
    #fiN = fi + z
    #if fiN > math.pi/2:
    #    fiN = math.pi - fiN
    
    # if check < 0 then trajectory crosses the pole
    # crossCheck = math.cos(z)*math.cos(fi) - math.sin(z)*math.sin(fi)
    
    # if fiMax > 0 trajectory crosses the pole
    fiMax = mat.fabs(fi) + mat.fabs(z)
    
    fiP = np.zeros(N)
    laP = np.zeros(N)
        
    x = mat.pi * np.linspace(0, 2, N)
    
    for i in range(N):
        fiP[i] = mat.asin( mat.sin(fi)*mat.cos(z) + mat.sin(z)*mat.cos(fi)*mat.sin(x[i]) )
        
        laN = mat.sin(z)*mat.cos(x[i]) # Numerator (števec)
        laD = mat.cos(z)*mat.cos(fi) - mat.sin(z)*mat.sin(fi)*mat.sin(x[i]) # Denominator (imenovalec)
        
        if mat.fabs(laD) < 1e-15:   # avoid division by zero
            if (laN>0) and (laD>0):
                laP[i] = mat.pi/2
            elif (laN>0) and (laD<0):
                laP[i] = -mat.pi/2
            elif (laN<0) and (laD<0):
                laP[i] = 3*mat.pi/2
            elif (laN<0) and (laD>0):
                laP[i] = -3*mat.pi/2
        else:
            laP[i] = mat.atan( laN/laD )
            
            # do check of correct interval
            if (laN>0) and (laD<0):
                laP[i] = mat.pi + laP[i]
            elif (laN<0) and (laD<0):
                laP[i] = mat.pi + laP[i]
            elif (laN<0) and (laD>0):
                #if crossCheck < 0: # passing pole
                if fiMax > 90: # passing pole
                    laP[i] = 2*mat.pi + laP[i]
                
        
        laP[i] += la
        laP[i] = mat.fmod(laP[i],2*mat.pi) # set to interval [0,2Pi)
        if laP[i] > mat.pi:
            laP[i] -= 2*mat.pi # set to iterval [-Pi,Pi]
        
    return nt.rad2deg(fiP), nt.rad2deg(laP)

In [ ]:
def get_pos_path(body,N):
    
    name = body[0]
    dec = body[1]
    gha = body[2]
    h = body[3]
    
    z = 90 - h
    sFi = dec
    if gha < 180:
        sLa = -gha
    else:
        sLa = 360 - gha
    print('{:}: sFi={:.5f}, sLa={:.5f}, z={:.5f}'.format(name.upper(), sFi, sLa, z))
    
    nFi, nLa = circle_pts(sFi, sLa, z, N)
    
    return sFi, sLa, z, nFi, nLa

In [ ]:
def plot_pair_data(stars,fix,check,N):

    dN = 10
    sN = len(stars)
    
    map = bmap.Basemap(projection='merc',
               llcrnrlat=-80, # lower left corner
               llcrnrlon=-180,
               urcrnrlat=80,  # upper right corner
               urcrnrlon=180,
               resolution='c') #c croud par defaut, l low , h high , f full
    
    map.drawmeridians(np.arange(0,360,30), linewidth=0.2)
    map.drawparallels(np.arange(-90,90,20), linewidth=0.2)
    map.drawmeridians([0], linewidth=0.4, color='black')
    map.drawparallels([0], linewidth=0.4, color='black')
    
    for si in range(sN):
        sFi, sLa, z, nFi, nLa = get_pos_path(stars[si],N)
        
        # plot body GP
        map.plot(sLa, sFi, marker='.', color='red', linewidth=0.25, markersize=2, latlon=True)

        # Plot Lat and Long
        for i in range(N-1):
            if mat.copysign(1,nLa[i]) == mat.copysign(1,nLa[i+1]): # avoid crossind datum line
                map.plot([nLa[i],nLa[i+1]],[nFi[i],nFi[i+1]], color='green', linewidth=0.25, latlon=True)
            elif mat.fabs(nLa[i]) + mat.fabs(nLa[i+1]) < 90: # plot crossing greenwich meridian
                map.plot([nLa[i],nLa[i+1]],[nFi[i],nFi[i+1]], color='green', linewidth=0.25, latlon=True)
    
    # plot fix position
    for p in fix:
        map.plot(p[1], p[0], marker='.', color='blue', linewidth=0.25, markersize=2, latlon=True)

    # plot our location
    # map.plot(check[1], check[0], marker='*', color='cyan', linewidth=0.25, markersize=0.5, latlon=True)

    mpl.savefig('direct_method.pdf')
    mpl.show()

In [ ]:
# convert Gha to Longitude

def gha2la(gha):
    
    if gha > mat.pi:
        la = 2*mat.pi - gha
    else:
        la = -gha
    
    return la

In [ ]:
# Returns Geographic Position (GP) of the star based on DEC and GHA
# input: declination, GHA
# output: fi, lambda

def getZFL(h, dec, gha):
    
    z = mat.pi/2 - h
    fi = dec
    if gha > mat.pi:
        la = 2*mat.pi - gha
    else:
        la = -gha
    
    return [z,fi,la]

In [ ]:
# Returns great circle (GC) distance between two positions
# - input: fi1, fi2, dl (delta lambda)
# - output: great circle distance in radians

def getDist(fi1,fi2,dl):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosD = mat.cos(psi1)*mat.cos(psi2) + mat.sin(psi1)*mat.sin(psi2)*mat.cos(dl)
    
    return mat.acos(cosD)

In [ ]:
#

def getPi1(z1,z2,D):
    
    cosPi1 = (mat.cos(z2) - mat.cos(z1)*mat.cos(D)) / (mat.sin(z1)*mat.sin(D))
    
    return mat.acos(cosPi1)

In [ ]:
#

def getW1(fi1,fi2,D):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosW1 = (mat.cos(psi2) - mat.cos(psi1)*mat.cos(D)) / (mat.sin(psi1)*mat.sin(D))
    
    return mat.acos(cosW1)

In [ ]:
#

def getPsiX(z1, fi1, Wx):
    
    psi1 = mat.pi/2 - fi1
    
    cosPsiX = mat.cos(psi1)*mat.cos(z1) + mat.sin(psi1)*mat.sin(z1)*mat.cos(Wx)
    
    return mat.acos(cosPsiX)

In [ ]:
#

def getDLx(z1, fi1, fix):
    
    psi1 = mat.pi/2 - fi1
    psix = mat.pi/2 - fix
    
    cosDL = (mat.cos(z1) - mat.cos(psi1)*mat.cos(psix)) / (mat.sin(psi1)*mat.sin(psix))
    dl = mat.acos(cosDL)

    return dl

In [ ]:
#

def calculatePosition(h1,dec1,gha1,h2,dec2,gha2,debug=0):

    if debug:
        print('gha1:', nt.rad2deg(gha1))
        print('gha2:', nt.rad2deg(gha2))
    
    # set P1 to be the western position
    dgha = gha2 - gha1
    queue = True
    if dgha > 0:
        if dgha < mat.pi:
            queue = False
    else:
        if dgha < -mat.pi:
            queue = False
        
    if queue:
        fi1 = dec1
        fi2 = dec2
        la1 = gha2la(gha1)
        la2 = gha2la(gha2)
        z1 = mat.pi/2 - h1
        z2 = mat.pi/2 - h2
    else:
        fi1 = dec2
        fi2 = dec1
        la1 = gha2la(gha2)
        la2 = gha2la(gha1)
        z1 = mat.pi/2 - h2
        z2 = mat.pi/2 - h1

    dla = la2 - la1
    if dla > mat.pi:
        dla = -(2*mat.pi - dla)
    elif dla < -mat.pi:
        dla = 2*mat.pi + dla

    if debug:
        print('fi1={:}'.format(nt.rad2deg(fi1)))
        print('la1={:}'.format(nt.rad2deg(la1)))
        print('z1={:}'.format(nt.rad2deg(z1)))
        print('fi2={:}'.format(nt.rad2deg(fi2)))
        print('la2={:}'.format(nt.rad2deg(la2)))
        print('z2={:}'.format(nt.rad2deg(z2)))
        print('dla={:}'.format(nt.rad2deg(dla)))

    D = getDist(fi1,fi2, dla)
    if debug: print('D:', nt.prettyPrintDM(nt.rad2deg(D),3))
    Pi1 = getPi1(z1,z2,D)
    if debug: print('Pi1:', nt.prettyPrintDM(nt.rad2deg(Pi1),3))
    W1 = getW1(fi1,fi2,D)
    if debug: print('W1:', nt.prettyPrintDM(nt.rad2deg(W1),3))
    
    alpha1N = W1 - Pi1
    alpha1S = W1 + Pi1
    if debug:
        print('a1N:', nt.prettyPrintDM(nt.rad2deg(alpha1N),3))
        print('a1S:', nt.prettyPrintDM(nt.rad2deg(alpha1S),3))

    psiXN = getPsiX(z1,fi1,alpha1N)
    if debug: print('psiXN:', nt.prettyPrintDM(nt.rad2deg(psiXN),3))
    fiXN = mat.pi/2 - psiXN
    dlXN = getDLx(z1,fi1,fiXN)
    if debug: print('dlaXN:', nt.prettyPrintDM(nt.rad2deg(dlXN),3))

    psiXS = getPsiX(z1,fi1,alpha1S)
    if debug: print('psiXS:', nt.prettyPrintDM(nt.rad2deg(psiXS),3))
    fiXS = mat.pi/2 - psiXS
    dlXS = getDLx(z1,fi1,fiXS)
    if debug: print('dlaXS:', nt.prettyPrintDM(nt.rad2deg(dlXS),3))
    
    if (alpha1N < 0) or (alpha1N > mat.pi):
        laXN = la1 - np.abs(dlXN)
    else:
        laXN = la1 + np.abs(dlXN)

    if (alpha1S < 0) or (alpha1S > mat.pi):
        laXS = la1 - np.abs(dlXS)
    else:
        laXS = la1 + np.abs(dlXS)
        

    pxN = [fiXN, laXN]
    pxS = [fiXS, laXS]
    
    return [pxN,pxS]

In [ ]:
def calculate_pair_position(cb1,cb2,debug=0):

    #print('cb1:', cb1)
    dec1 = nt.deg2rad(cb1[1])
    gha1 = nt.deg2rad(cb1[2])
    h1 = nt.deg2rad(cb1[3])

    #print('cb2:', cb2)
    dec2 = nt.deg2rad(cb2[1])
    gha2 = nt.deg2rad(cb2[2])
    h2 = nt.deg2rad(cb2[3])

    [PxN, PxS] = calculatePosition(h1,dec1,gha1,h2,dec2,gha2,debug)

    return [[nt.rad2deg(PxN[0]), nt.rad2deg(PxN[1])],[nt.rad2deg(PxS[0]), nt.rad2deg(PxS[1])]]

In [ ]:
def test_cb_01(first,second):

    #Alioth       h = 34°55.18′  ω = 043.21°  dec = 55°49.55′  Gha = 282°11.06′
    name1 = 'Alioth'
    h1 = [34,55.18]; h1_rad = nt.deg2rad(nt.dms2dd(h1))
    dec1 = [55,49.55,'N']; dec1_rad = nt.deg2rad(nt.nav2dd(dec1))
    gha1 = [282,11.06]; gha1_rad = nt.deg2rad(nt.dms2dd(gha1))
    cb1 = [name1, nt.nav2dd(dec1), nt.dms2dd(gha1), nt.dms2dd(h1)]

    #Sirius       h = 28°14.81′  ω = 180.45°  dec = -16°45.12′  Gha = 14°24.79′
    name2 = 'Sirius'
    h2 = [28,14.81]; h2_rad = nt.deg2rad(nt.dms2dd(h2))
    dec2 = [16,45.12,'S']; dec2_rad = nt.deg2rad(nt.nav2dd(dec2))
    gha2 = [14,24.79]; gha2_rad = nt.deg2rad(nt.dms2dd(gha2))
    cb2 = [name2, nt.nav2dd(dec2), nt.dms2dd(gha2), nt.dms2dd(h2)]

    #Hamal        h = 30°24.64′  ω = 273.95°  dec = 23°34.56′  Gha = 83°50.30′
    name3 = 'Hamal'
    h3 = [30,24.64]; h3_rad = nt.deg2rad(nt.dms2dd(h3))
    dec3 = [23,34.56,'N']; dec3_rad = nt.deg2rad(nt.nav2dd(dec3))
    gha3 = [83,50.3]; gha3_rad = nt.deg2rad(nt.dms2dd(gha3))
    cb3 = [name3, nt.nav2dd(dec3), nt.dms2dd(gha3), nt.dms2dd(h3)]

    if first == 1 and second == 2:
        return [[cb1,h1_rad,dec1_rad,gha1_rad],[cb2,h2_rad,dec2_rad,gha2_rad]]
    elif first == 1 and second == 3:
        return [[cb1,h1_rad,dec1_rad,gha1_rad],[cb3,h3_rad,dec3_rad,gha3_rad]]
    else:
        return [[cb3,h3_rad,dec3_rad,gha3_rad],[cb2,h2_rad,dec2_rad,gha2_rad]]

In [ ]:
def test_cb_02(first,second):

    #Dubhe        h = 50°08.48′  ω = 041.95°  dec = 61°37.23′  Gha = 309°39.19′
    name1 = 'Dubhe'
    h1 = [50,8.48]; h1_rad = nt.deg2rad(nt.dms2dd(h1))
    dec1 = [61,37.23,'N']; dec1_rad = nt.deg2rad(nt.nav2dd(dec1))
    gha1 = [309,39.19]; gha1_rad = nt.deg2rad(nt.dms2dd(gha1))
    cb1 = [name1, nt.nav2dd(dec1), nt.dms2dd(gha1), nt.dms2dd(h1)]

    #Pollux       h = 69°19.60′  ω = 140.49°  dec = 27°58.10′  Gha = 359°15.99′
    name2 = 'Pollux'
    h2 = [69,19.6]; h2_rad = nt.deg2rad(nt.dms2dd(h2))
    dec2 = [27,58.1,'N']; dec2_rad = nt.deg2rad(nt.nav2dd(dec2))
    gha2 = [359,15.99]; gha2_rad = nt.deg2rad(nt.dms2dd(gha2))
    cb2 = [name2, nt.nav2dd(dec2), nt.dms2dd(gha2), nt.dms2dd(h2)]

    #Mirfak       h = 56°11.71′  ω = 296.89°  dec = 49°56.94′  Gha = 64°27.56′
    name3 = 'Mirfak'
    h3 = [56,11.71]; h3_rad = nt.deg2rad(nt.dms2dd(h3))
    dec3 = [49,56.94,'N']; dec3_rad = nt.deg2rad(nt.nav2dd(dec3))
    gha3 = [64,27.56]; gha3_rad = nt.deg2rad(nt.dms2dd(gha3))
    cb3 = [name3, nt.nav2dd(dec3), nt.dms2dd(gha3), nt.dms2dd(h3)]

    if first == 1 and second == 2:
        return [[cb1,h1_rad,dec1_rad,gha1_rad],[cb2,h2_rad,dec2_rad,gha2_rad]]
    elif first == 1 and second == 3:
        return [[cb1,h1_rad,dec1_rad,gha1_rad],[cb3,h3_rad,dec3_rad,gha3_rad]]
    else:
        return [[cb3,h3_rad,dec3_rad,gha3_rad],[cb2,h2_rad,dec2_rad,gha2_rad]]

In [ ]:
# observer position (what should we get, for control)!!
fi_check = [45,0,'N'];  fi_check_deg = nt.nav2dd(fi_check)
la_check = [14,0,'W']; la_check_deg = nt.nav2dd(la_check)

# Inputs:
first = 2
second = 3

case = 2 # case can be 1 or 2
if case == 1:
    # Case - 01
    [[cb1,h1_rad,dec1_rad,gha1_rad],[cb2,h2_rad,dec2_rad,gha2_rad]] = test_cb_01(first,second)
else:
    # Case - 02
    [[cb1,h1_rad,dec1_rad,gha1_rad],[cb2,h2_rad,dec2_rad,gha2_rad]] = test_cb_02(first,second)

[z1,fi1,la1] = getZFL(h1_rad,dec1_rad,gha1_rad)
print('*Star 1* - GP and zenith distance:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(nt.rad2deg(fi1))))
print('  -> la = {:}'.format(nt.prettyPrintLong(nt.rad2deg(la1))))
print('  ->  z =  {:} (observed)'.format(nt.prettyPrintAlt(nt.rad2deg(z1))))
print()

[z2,fi2,la2] = getZFL(h2_rad,dec2_rad,gha2_rad)
print('*Star 2* - GP and zenith distance:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(nt.rad2deg(fi2))))
print('  -> la = {:}'.format(nt.prettyPrintLong(nt.rad2deg(la2))))
print('  ->  z =  {:} (observed)'.format(nt.prettyPrintAlt(nt.rad2deg(z2))))
print()

print('-----------------------------------------')
print('Calculation parameters:')
[PxN, PxS] = calculate_pair_position(cb1,cb2,0)
print('-----------------------------------------')
print()

print('*** Calculated position: ***')
print('North')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(PxN[0])))
print('  -> la = {:}'.format(nt.prettyPrintLong(PxN[1])))
print()
print('South:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(PxS[0])))
print('  -> la = {:}'.format(nt.prettyPrintLong(PxS[1])))
print()
print('Error in calculated position')
print('North')
print('  -> fi = {:.3f} Nm'.format(np.abs(fi_check_deg-PxN[0])*60))
print('  -> la = {:.3f} Nm'.format(np.abs(la_check_deg-PxN[1])*60))
print()
print('South:')
print('  -> fi = {:.3f} Nm'.format(np.abs(fi_check_deg-PxS[0])*60))
print('  -> la = {:.3f} Nm'.format(np.abs(la_check_deg-PxS[1])*60))

print()
print('--------------------------')
cbs = [cb1, cb2]
cb_fix = calculate_pair_position(cb1,cb2)
check_fix = [fi_check_deg, la_check_deg]
plot_pair_data(cbs,cb_fix,check_fix,100)